In [1]:
import gc

import duckdb
from transformers import BigBirdModel, BigBirdTokenizerFast
from textsum.summarize import Summarizer

2023-10-11 22:36:29.166428: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Conectar (ou criar) um banco de dados
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [ ]:
con.execute("CREATE TABLE metadadosPublicacao202202 AS SELECT * FROM read_json_auto('./data/202202-stj/metadadosPublicacao202202.json');")

In [ ]:
con.execute("DELETE FROM metadadosPublicacao202202 WHERE teor NOT IN ('Concedendo', 'Negando') or teor IS NULL;")

In [3]:
con.execute("select * from metadadosPublicacao202202").df()

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos
0,144948780,1644458400000,ACÓRDÃO,202002151590,REsp 1890871,1598238000000,1598929200000,ASSUSETE MAGALHÃES,None,Concedendo,None,10254;10225;10254;10225
1,145620297,1645412400000,ACÓRDÃO,202103074377,HC 695900,1632279600000,1632279600000,SEBASTIÃO REIS JÚNIOR,None,Concedendo,None,3372;3372
2,139579332,1644890400000,ACÓRDÃO,202103073420,HC 695866,1632279600000,1632279600000,ANTONIO SALDANHA PALHEIRO,None,Concedendo,None,4355;4355;3608
3,141816757,1645758000000,ACÓRDÃO,202103076854,Rcl 42334,1632279600000,1632452400000,REYNALDO SOARES DA FONSECA,None,Concedendo,None,3608;3608
4,144148942,1645412400000,ACÓRDÃO,202103167750,HC 697730,1632884400000,1633057200000,OLINDO MENEZES (DESEMBARGADOR CONVOCADO DO TRF...,None,Concedendo,None,3608;3608
...,...,...,...,...,...,...,...,...,...,...,...,...
35399,144531393,1643940000000,DECISÃO,201100020306,REsp 1228590,1294192800000,1296007200000,OG FERNANDES,EMBARGOS DE DECLARAÇÃO,Concedendo,Embargos de Declaração Acolhidos,10318
35400,143092827,1645758000000,DECISÃO,201001614248,REsp 1210458,1285642800000,1288058400000,OG FERNANDES,None,Concedendo,Conhecido em parte o recurso e provido em parte,7760
35401,142924179,1643767200000,DECISÃO,200902062079,EREsp 1163268,1256004000000,1256608800000,FRANCISCO FALCÃO,None,Concedendo,Conhecido o recurso e provido,10028;10028;1000238
35402,145440916,1644976800000,DECISÃO,200801470759,AR 3998,1214794800000,1214794800000,BENEDITO GONÇALVES,EMBARGOS DE DECLARAÇÃO,Concedendo,Embargos de Declaração Acolhidos,8826


In [ ]:
# tabela de texto dos juiz
con.execute("""
  CREATE TABLE decisions_description (
    ID INT PRIMARY KEY,
    content TEXT,
    summarize TEXT
  );
""")

In [ ]:
con.execute("COPY decisions_description FROM './decisions_description_202310102238.csv' DELIMITER ',' HEADER;")

In [3]:
con.execute("select * from decisions_description").df()

,ID,content,summarize
0,121184492,DECISION > It is treated of habeas corpus impe...,Gulliver argues that the prison system is inhe...
1,122027196,DECISION<br> It is considered a special appeal...,"In this special appeal, a citizen appeals agai..."
2,123312824,DECISION<br> It is considered an ordinary appe...,the author defends an ordinary appeal brought ...
3,123447928,DECISION<br> It is treated of habeas corpus im...,Gulliver appeals to the court for a partial re...
4,123917739,DECISION<br> It is considered a special appeal...,"In this special appeal, the plaintiff, Nichole..."
...,...,...,...
5800,146076514,DECISION<br> By understanding that better exam...,None
5801,144454767,DECISION<br> By understanding that better exam...,None
5802,145089835,DECISION<br> By understanding that better exam...,None
5803,145511648,DECISION<br> By understanding that better exam...,None


In [9]:
con.execute("CREATE TABLE BigBird(id INT PRIMARY KEY, data FLOAT8[]);")

In [8]:
con.execute("drop table BigBird")

In [10]:
con.execute("CREATE TABLE BigBird_norm(id INT UNIQUE REFERENCES BigBird(id), norm FLOAT8);")

In [7]:
con.execute("drop table BigBird_norm")

In [11]:
summarizer = Summarizer(
    model_name_or_path="pszemraj/long-t5-tglobal-base-16384-book-summary",
    num_beams=4,
    force_cache=True,
    #optimum_onnx=True,
    compile_model=True
)

10/10/2023 11:12:28 INFO Compiling model
10/10/2023 11:12:30 INFO Loaded model pszemraj/long-t5-tglobal-base-16384-book-summary to cpu
10/10/2023 11:12:30 INFO Forcing use_cache to True


In [12]:
tokenizer = BigBirdTokenizerFast.from_pretrained(
    "google/bigbird-roberta-base")  # large
model = BigBirdModel.from_pretrained("google/bigbird-roberta-base",
                                     attention_type="original_full"
                                     # attention_type="block_sparse"
                                     )

In [13]:
decisions_description = con.execute("SELECT ID,content FROM decisions_description").df()
decisions_description

,ID,content
0,121184492,DECISION > It is treated of habeas corpus impe...
1,122027196,DECISION<br> It is considered a special appeal...
2,123312824,DECISION<br> It is considered an ordinary appe...
3,123447928,DECISION<br> It is treated of habeas corpus im...
4,123917739,DECISION<br> It is considered a special appeal...
...,...,...
5800,146076514,DECISION<br> By understanding that better exam...
5801,144454767,DECISION<br> By understanding that better exam...
5802,145089835,DECISION<br> By understanding that better exam...
5803,145511648,DECISION<br> By understanding that better exam...


In [14]:
for index, content in enumerate(decisions_description["content"]):
  if index >= 0 and index <= 30:
    out_str = summarizer.summarize_string(content)
    seqDocumento = int(decisions_description["ID"][index])

    con.execute("""
      UPDATE decisions_description
      SET summarize = $out_str
      WHERE ID = $id;
      """,{
        'out_str': out_str,
        'id': seqDocumento,
      })
    
    inputs = tokenizer(out_str, return_tensors="pt")
    outputs = model(**inputs)

    print(index,seqDocumento,len(content),len(out_str),outputs.last_hidden_state[0].shape)

    vetor = outputs.last_hidden_state[0].flatten().detach().numpy()
    con.execute('INSERT INTO BigBird (id,data) VALUES (?,?)', (seqDocumento,vetor))

con.commit()

Generating Summaries:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jadson/anaconda3/envs/py/lib/python3.10/site-packages/transformers/modeling_utils.py:835: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


0 121184492 1102
0 121184492 1102 torch.Size([211, 768])


Generating Summaries:   0%|          | 0/2 [00:00<?, ?it/s]

1 122027196 1050
1 122027196 1050 torch.Size([219, 768])


...

28 133275190 724
28 133275190 724 torch.Size([139, 768])


Generating Summaries:   0%|          | 0/2 [00:00<?, ?it/s]

29 133344745 699
29 133344745 699 torch.Size([140, 768])


Generating Summaries:   0%|          | 0/1 [00:00<?, ?it/s]

30 134122233 263
30 134122233 263 torch.Size([55, 768])


In [7]:
con.execute("""
WITH norms AS (
    SELECT 
        id,
        SQRT(SUM(POWER(val, 2))) AS norm
    FROM 
    (
        SELECT
            id,
            UNNEST(data) AS val
        FROM 
            BigBird
    ) AS subquery
    GROUP BY id
)

INSERT INTO BigBird_norm (id, norm)
SELECT id, norm
FROM norms
ON CONFLICT (id) 
DO UPDATE SET norm = EXCLUDED.norm;

""")

In [8]:
con.execute("SELECT * FROM BigBird_norm").df()

,id,norm
0,127506614,344.751405
1,128070511,104.631807
2,130072596,212.710395
3,130527099,125.903486
4,131460066,236.974861
5,129636667,547.138103
6,130223439,192.474233
7,121184492,168.704783
8,124072953,144.833634
9,129980126,181.511283


In [9]:
con.execute("""
WITH DotProducts AS (
    WITH expanded AS (
        SELECT
            a.id AS idA,
            b.id AS idB,
            unnest(a.data) AS a_data,
            unnest(b.data) AS b_data,
            row_number() OVER(PARTITION BY a.id, b.id ORDER BY a.id) as rn
        FROM
            BigBird a
        CROSS JOIN
            BigBird b
        WHERE
            a.id < b.id
    )

    SELECT
        idA,
        idB,
        sum(a_data * b_data) AS dot_product
    FROM
        expanded
    GROUP BY
        idA, idB
                
    )
SELECT 
    d.idA,
    d.idB,
    CASE 
        WHEN n1.norm = 0 OR n2.norm = 0 THEN 0
        ELSE d.dot_product / (n1.norm * n2.norm)
    END AS cosine_similarity
FROM 
    DotProducts d
JOIN 
    BigBird_norm n1 ON d.idA = n1.id
JOIN 
    BigBird_norm n2 ON d.idB = n2.id;
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,idA,idB,cosine_similarity
0,121184492,132206882,0.306445
1,121184492,133275190,0.404959
2,121184492,133344745,0.397468
3,122027196,128070511,0.297996
4,123312824,127506614,0.229289
...,...,...,...
460,131460066,133344745,0.252563
461,127181699,130933413,0.175628
462,127550136,130933413,0.238654
463,130148392,130933413,0.343294


In [10]:
gc.collect()

1054

In [11]:
con.commit()

In [11]:
con.commit()
con.close()